### Getting started - bring in TF and other dependencies for data loading
Some of these dependencies I pulled from hangyao's deep_learning repository on github - in particular, the data ingest process.

In [4]:
import tensorflow as tf
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import os
import sys
import tarfile
from IPython.display import display, Image
from six.moves.urllib.request import urlretrieve
from six.moves import cPickle as pickle

In [5]:
%matplotlib inline